In [1]:
#필요 모듈들
import cv2
import os
from keras.models import load_model
import numpy as np

#image test하는 파일
def image_test(image_path):
        #opencv에서 받아올 수 있는 haar cascade (기존에 학습된 파일)
    face = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
    # 얼굴 검출기
    leye = cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
    # 왼쪽 눈 검출
    reye = cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')
    # 오른쪽 눈 검출
    
    lbls = ['Close','Open']
    
    model=load_model('models/mytest.h5')
    image=cv2.imread(image_path)
    font = cv2.FONT_HERSHEY_COMPLEX_SMALL
    height,width = image.shape[:2] 

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    count=0
    #score=0
    #thicc=2
    rpred=[99]
    lpred=[99]
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(image, (x,y) , (x+w,y+h) , (0,0,255) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=image[y:y+h,x:x+w]
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict_classes(r_eye)
            
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=image[y:y+h,x:x+w]
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = model.predict_classes(l_eye)
            #left_processed_image = np.stack([cv2.resize(preprocess_input(img.astype(np.float32)),(24,24))],axis=0)
            #prediction1 = model.predict(left_processed_image)
            
        if(lpred[0]==1):
            lbl='Open'   
        if(lpred[0]==0):
            lbl='Closed'
        break
        
    if(rpred[0]==0 and lpred[0]==0):
        cv2.putText(image,"Closed",(10,height-20), font, 1,(0,0,255),1,cv2.LINE_AA)
    else:
        cv2.putText(image,"Open",(10,height-20), font, 1,(0,0,255),1,cv2.LINE_AA)
        
        #if cv2.waitKey(1) & 0xFF == ord('q'):
            #break
    cv2.imshow("test",image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    
    
            
            
    

Using TensorFlow backend.


In [7]:
image_test("openimage.jpg")

In [6]:
image_test("closedimage.jpg")